# Libraries 📚

In [2]:
import glob
import json
import numpy as np

/Users/dhebel/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Obtain files 🗂️

In [2]:
files_MUM_03 = glob.glob('translations/0.3/mu-/*.json')
files_MUP_03 = glob.glob('translations/0.3/mu+/*.json')
files_MUM_01 = glob.glob('translations/0.1/mu-/*.json')
files_MUP_01 = glob.glob('translations/0.1/mu+/*.json')
files_NEU_03 = glob.glob('translations/0.3/neutron/*.json')

# Building X and y matrices

In [ ]:
def reconstruct_matrix(filelist):
        # The matrix will be filled with the energy deposited in each sparse coordinate (same coordinates as in the matrix).
        # The first channel is filled with the energy from the hits of the side 1 and the second channel is filled with the energy from hits of the side 2.
        
        X = []
        
        for file in filelist:
                with open(file) as f:
                        data = json.load(f)

                for event, event_data in data.items():
                        side_1 = event_data['gruid hits - side 1']
                        side_2 = event_data['gruid hits - side 2']
                        gruid_metadata = event_data['gruid metadata']
                        
                matrix = np.zeros((gruid_metadata['# of rows (y)'], gruid_metadata['# of columns (x)'], 2))
        
                # IMPORTANT: Coordinates are inverted in the matrix because of matrix notation in GRUID.
                        
                # Side 1
                for timestamp, timestamp_data in side_1.items():
                        for pixel, pixel_data in timestamp_data.items():
                                x, y = pixel.split(",")
                                matrix[ int(y), int(x), 0 ] = pixel_data['energy deposited']

                # Side 2
                for timestamp, timestamp_data in side_2.items():
                        for pixel, pixel_data in timestamp_data.items():
                                x, y = pixel.split(",")
                                matrix[ int(y), int(x), 1 ] = pixel_data['energy deposited']
                                                
                X.append(matrix)
                
        return np.array(X)
     
   
# 0.3 --------------------------------------------
muon_labels = np.ones(len(files_MUM_03))
antimuon_labels = np.zeros(len(files_MUP_03))
y_03 = np.concatenate((muon_labels, antimuon_labels))

muons = reconstruct_matrix(files_MUM_03)
antimuons = reconstruct_matrix(files_MUP_03)
X_03 = np.concatenate((muons, antimuons))

# 0.1 --------------------------------------------
muon_labels = np.ones(len(files_MUM_01))
antimuon_labels = np.zeros(len(files_MUP_01))
y_01 = np.concatenate((muon_labels, antimuon_labels))

muons = reconstruct_matrix(files_MUM_01)
antimuons = reconstruct_matrix(files_MUP_01)
X_01 = np.concatenate((muons, antimuons))

neutrons = reconstruct_matrix(files_NEU_03)
neutron_labels = np.ones(len(files_NEU_03))*2

## Saving Matrices to files

In [5]:
# Save matrices to file
np.save('matrices/X_03.npy', X_03)
np.save('matrices/y_03.npy', y_03)
np.save('matrices/X_01.npy', X_01)
np.save('matrices/y_01.npy', y_01)
np.save('matrices/nX.npy', neutrons)
np.save('matrices/ny.npy', neutron_labels)